In [2]:
!pip install openvino

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.8/33.8 MB 3.7 MB/s eta 0:00:0000:0100:01


In [3]:
import sys
sys.path.append("..")
import torch
from models.CQTNet import CQTNet, CQTNetSetlist
import openvino as ov
import os

# Convert CQTNet

In [7]:
CQTNET_MODEL_PATH = "check_points/CQTNet.pth"
CQTNET_OV_MODEL_PATH = "/home/lennon/workspace/CQTNet/check_points/openvino/CQTNET.xml"

In [8]:
cqtnet_model = CQTNet()
cqtnet_model.load_state_dict(torch.load(CQTNET_MODEL_PATH, map_location=torch.device('cpu')))
cqtnet_model.eval()
ov_model_cqtnet = ov.convert_model(cqtnet_model)

In [9]:
if not os.path.isfile(CQTNET_OV_MODEL_PATH):
    ov.save_model(ov_model_cqtnet, CQTNET_OV_MODEL_PATH)
    print("OpenVino model created")
else:
    print("OpenVino model already exists")

OpenVino model created


# Convert CQTNet Finetuned

In [4]:
CQTNET_MODEL_PATH = "check_points/finetuning/latest.pth"
CQTNET_OV_MODEL_PATH = "check_points/openvino/CQTNET_v2.xml"

In [5]:
cqtnet_model = CQTNetSetlist() #CQTNet()
cqtnet_model.load_state_dict(torch.load(CQTNET_MODEL_PATH, map_location=torch.device('cpu')))
cqtnet_model.eval()
ov_model_cqtnet = ov.convert_model(cqtnet_model)

In [6]:
if not os.path.isfile(CQTNET_OV_MODEL_PATH):
    ov.save_model(ov_model_cqtnet, CQTNET_OV_MODEL_PATH)
    print("OpenVino model created")
else:
    print("OpenVino model already exists")

OpenVino model created
